In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

# 1: PROCESS GUN VIOLENCE DATA AND ADD COLUMNS #

In [ ]:
gunvDF = pd.read_csv("gun-violence-data_01-2013_03-2018.csv")

In [ ]:
gunvDF.to_csv("gun-violence-data_PROCESSED.csv", index=False)

## *A: Add State Abbreviations* ##

In [ ]:
gunvDF.insert(3, "state_abv", gunvDF["state"])

In [ ]:
stateabrDF = pd.read_csv("census_votingAgeSexPoverty.csv")
stateabrDF = stateabrDF[["state_abv", "state"]]
stateabrDF.drop_duplicates(inplace=True)
stateabrDF.to_csv("stateAbreviations.csv", index=False)

In [ ]:
lookup = pd.DataFrame({'label': ["AL","AK","AZ","AR","CA","CO","CT","DE","DC","FL","GA","HI","ID","IL","IN","IA", \
                                "KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC", \
                                "ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"],
                        'id': ["Alabama","Alaska","Arizona","Arkansas","California","Colorado","Connecticut","Delaware", \
                                "District of Columbia","Florida","Georgia","Hawaii","Idaho","Illinois","Indiana","Iowa","Kansas", \
                                "Kentucky","Louisiana","Maine","Maryland","Massachusetts","Michigan","Minnesota","Mississippi", \
                                "Missouri","Montana","Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York","North Carolina", \
                                "North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania","Rhode Island","South Carolina","South Dakota", \
                                "Tennessee","Texas","Utah","Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]
                        })

In [ ]:
gunvDF["state_abv"].replace(dict(zip(lookup.id, lookup.label)), inplace=True)

## *B: Add Concat State+District* ##

In [ ]:
gunvDF["congressional_district"].fillna(0, inplace=True)

In [ ]:
gunvDF["congressional_district"] = gunvDF["congressional_district"].astype(int)
gunvDF.loc[gunvDF.congressional_district == 0, "congressional_district"] = "UNK"
gunvDF["congressional_district"] = gunvDF["congressional_district"].astype(str)

In [ ]:
gunvDF.loc[gunvDF.state_abv == "AK", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "DE", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "DC", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "MT", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "ND", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "SD", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "VT", "congressional_district"] = "At-large"
gunvDF.loc[gunvDF.state_abv == "WY", "congressional_district"] = "At-large"

In [ ]:
gunvDF.insert(4, "state_house_dist", gunvDF["state_abv"]+"-"+(gunvDF["congressional_district"].map(str)))

# 2: PROCESS AND COMBINE CENSUS DATA #

In [ ]:
eduDF = pd.read_csv('census_votingAgeEducation.csv')
ageDF = pd.read_csv('census_votingAgePopulation.csv')
raceDF = pd.read_csv('census_votingAgeRace.csv')
sex_povDF = pd.read_csv('census_votingAgeSexPoverty.csv')
voteRatesDF = pd.read_csv('census_votingPopulationRates.csv')

In [ ]:
eduDF.insert(4, "state_house_dist", eduDF["state_abv"]+"-"+(eduDF["house_district"].map(str)))
ageDF.insert(4, "state_house_dist", ageDF["state_abv"]+"-"+(ageDF["house_district"].map(str)))
raceDF.insert(4, "state_house_dist", raceDF["state_abv"]+"-"+(raceDF["house_district"].map(str)))
sex_povDF.insert(4, "state_house_dist", sex_povDF["state_abv"]+"-"+(sex_povDF["house_district"].map(str)))
voteRatesDF.insert(4, "state_house_dist", voteRatesDF["state_abv"]+"-"+(voteRatesDF["house_district"].map(str)))

In [ ]:
census_MASTER = pd.concat([
    eduDF['state_abv'],eduDF['state_house_dist'],eduDF['edu_votingage_est'],eduDF['edu_ltgrade9_perc'],eduDF['edu_9to12_perc'],eduDF['edu_hs_perc']
    ,eduDF['edu_somecollege_perc'],eduDF['edu_bachelor_perc'],eduDF['edu_graduate_perc'],eduDF['edu_hsormore_perc'],eduDF['edu_bachormore_perc']
    ,ageDF['state_abv'],ageDF['state_house_dist'],ageDF['pop_votingage_est'],ageDF['age_18to29_perc'],ageDF['age_30to44_perc'],ageDF['age_45to64_perc'],ageDF['age_65plus_perc']
    ,raceDF['state_abv'],raceDF['state_house_dist'],raceDF['pop_votingage_est'],raceDF['race_white_perc'],raceDF['race_black_perc'],raceDF['race_asian_perc']
    ,raceDF['race_native_perc'],raceDF['race_pacisland_perc'],raceDF['race_otherrace_perc'],raceDF['race_mixedrace_perc'],raceDF['race_hispanic_perc'],raceDF['race_wtnonhispanic_perc']
    ,sex_povDF['state_abv'],sex_povDF['state_house_dist'],sex_povDF['pop_votingage_est'],sex_povDF['male_perc']
    ,sex_povDF['female_perc'],sex_povDF['povertyuniverse_est'],sex_povDF['poverty_perc']
    ,voteRatesDF['state_abv'],voteRatesDF['state_house_dist'],voteRatesDF['pop_votingage_est'],voteRatesDF['votes_casts'],voteRatesDF['pop_votingrate_perc']
    ], axis=1)

In [ ]:
census_MASTER.to_csv("census_MASTER.csv", index=False)

# 3: PROCESS AND COMBINE REPRESENTATIVE DATA #

In [ ]:
DF108 = pd.read_csv("108-2003-01-06_2005-01-03.csv")
DF109 = pd.read_csv("109-2005-01-06_2007-01-03.csv")
DF110 = pd.read_csv("110-2007-01-06_2009-01-03.csv")
DF111 = pd.read_csv("111-2009-01-06_2011-01-03.csv")
DF112 = pd.read_csv("112-2011-01-06_2013-01-03.csv")
DF113 = pd.read_csv("113-2013-01-06_2015-01-03.csv")
DF114 = pd.read_csv("114-2015-01-06_2017-01-03.csv")
DF115 = pd.read_csv("115-2017-01-03_2019-01-03.csv")

In [ ]:
representatives_MASTER = pd.concat([DF108, DF109, DF110, DF111, DF112, DF113, DF114, DF115])

In [ ]:
representatives_MASTER = representatives_MASTER.drop(['id', 'sort_name', 'email', 'twitter', 'facebook',
       'group_id', 'area', 'chamber', 'image', 'gender', 'wikidata', 'wikidata_group', 'wikidata_area'], axis=1)

In [ ]:
representatives_MASTER.loc[(representatives_MASTER["term"]==108), "termStart"] = "2003-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==108), "termEnd"] = "2005-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==109), "termStart"] = "2005-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==109), "termEnd"] = "2007-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==110), "termStart"] = "2007-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==110), "termEnd"] = "2009-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==111), "termStart"] = "2009-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==111), "termEnd"] = "2011-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==112), "termStart"] = "2011-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==112), "termEnd"] = "2013-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==113), "termStart"] = "2013-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==113), "termEnd"] = "2015-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==114), "termStart"] = "2015-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==114), "termEnd"] = "2017-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==115), "termStart"] = "2017-JAN"
representatives_MASTER.loc[(representatives_MASTER["term"]==115), "termEnd"] = "2019-JAN"

In [ ]:
representatives_MASTER.to_csv("representatives_MASTER.csv", index=False)

# 4: COMBINE VIOLENCE AND CENSUS DATA BY DISTRICT #

In [ ]:
gunvDF = pd.read_csv("gun-violence-data_PROCESSED.csv")
censusDF = pd.read_csv("census_MASTER.csv")

In [ ]:
# DROP YEAR 2013 -- HAS LESS THAN 200 RECORDS WHICH IS NOT ACCURATE
# DROP YEAR 2018 -- HAS ONLY 3 MONTHS OF DATA

gunvDF[["year", "month", "day"]] = gunvDF["date"].str.split('-', expand=True)
gunvDF.drop(gunvDF[gunvDF.year == "2013"].index, inplace=True)
gunvDF.drop(gunvDF[gunvDF.year == "2018"].index, inplace=True)
gunvDF.drop(['date', 'state', 'state_abv', 'city_or_county', 'address', 'n_killed', 'n_injured', 'incident_url',
       'source_url', 'incident_url_fields_missing', 'congressional_district',
       'gun_stolen', 'gun_type', 'incident_characteristics', 'latitude',
       'location_description', 'longitude', 'n_guns_involved', 'notes',
       'participant_age', 'participant_age_group', 'participant_gender',
       'participant_name', 'participant_relationship', 'participant_status',
       'participant_type', 'sources', 'state_house_district',
       'state_senate_district'], axis=1, inplace=True)

In [ ]:
grouped = gunvDF.groupby(["state_house_dist"]).count().reset_index()
grouped.drop(["year", "month", "day"], axis=1, inplace=True)
grouped.rename({"incident_id": "count_of_inc"}, axis=1, inplace=True)

In [ ]:
vio_censusDF = grouped.join(censusDF.set_index("state_house_dist"), on="state_house_dist")

In [ ]:
vio_censusDF.columns

In [ ]:
vio_censusDF.drop(['edu_votingage_est', 'state_abv.1', 'state_house_dist.1', 'state_abv.2',
       'state_house_dist.2', 'pop_votingage_est.1', 'state_abv.3',
       'state_house_dist.3', 'pop_votingage_est.2', 'state_abv.4',
       'state_house_dist.4', 'pop_votingage_est.3'], axis=1, inplace=True)

In [ ]:
vio_censusDF.to_csv("violence_census_MASTER.csv", index=False)